This notebook explores WordNet synsets, presenting a simple method for finding in a text all mentions of all hyponyms of a given node in the WordNet hierarchy (e.g., finding all *buildings* in a text).

In [1]:
import nltk, re, spacy
from nltk.corpus import wordnet as wn

In [2]:
nlp = spacy.load('en_core_web_sm', disable=['ner,parser'])
nlp.remove_pipe('ner')
nlp.remove_pipe('parser');

Get the synsets for a given word.  The synsets here are roughly ordered by frequency of use (in a small tagged dataset), so that more frequent senses occur first.

In [3]:
synsets=wn.synsets('blue')
for synset in synsets:
    print (synset, synset.definition())

Synset('blue.n.01') blue color or pigment; resembling the color of the clear sky in the daytime
Synset('blue.n.02') blue clothing
Synset('blue.n.03') any organization or party whose uniforms or badges are blue
Synset('blue_sky.n.01') the sky as viewed during daylight
Synset('bluing.n.01') used to whiten laundry or hair or give it a bluish tinge
Synset('amobarbital_sodium.n.01') the sodium salt of amobarbital that is used as a barbiturate; used as a sedative and a hypnotic
Synset('blue.n.07') any of numerous small butterflies of the family Lycaenidae
Synset('blue.v.01') turn blue
Synset('blue.s.01') of the color intermediate between green and violet; having a color similar to that of a clear unclouded sky; - Helen Hunt Jackson
Synset('blue.s.02') used to signify the Union forces in the American Civil War (who wore blue uniforms)
Synset('gloomy.s.02') filled with melancholy and despondency
Synset('blasphemous.s.02') characterized by profanity or cursing
Synset('blue.s.05') suggestive of 

Get the words/phrases in that synset.

In [4]:
for lemma in wn.synset("blue.n.01").lemmas():
    print (lemma.name())

blue
blueness


In [5]:
# Functions from http://www.nltk.org/howto/wordnet.html to get *all* of a synset's hyponym/hypernyms
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()

Find all of the synsets that are hyponyms of the target synset (*descendents* in the WordNet hierarchy)

In [6]:
list(wn.synset("blue.n.01").closure(hypo))

[Synset('azure.n.01'),
 Synset('dark_blue.n.01'),
 Synset('greenish_blue.n.01'),
 Synset('powder_blue.n.01'),
 Synset('prussian_blue.n.02'),
 Synset('purplish_blue.n.01'),
 Synset('steel_blue.n.01'),
 Synset('ultramarine.n.02')]

Find all of the synsets that are hyperyms (*ancestors* up the tree) of the target synset

In [7]:
list(wn.synset("blue.n.01").closure(hyper))

[Synset('chromatic_color.n.01'),
 Synset('color.n.01'),
 Synset('visual_property.n.01'),
 Synset('property.n.02'),
 Synset('attribute.n.02'),
 Synset('abstraction.n.06'),
 Synset('entity.n.01')]

In [8]:
def get_words_in_hypo(synset):
    """ Returns a list of words/phrases that comprise the hyponyms of a synset. 
    """
    words=set()
    hyponym_synsets=list(synset.closure(hypo))
    hyponym_synsets.append(synset)
    for synset in hyponym_synsets:
        for l in synset.lemmas():
            word=l.name()
            word=re.sub("_", " ", word)
            words.add(word)
    
    return words

In [9]:
get_words_in_hypo(wn.synset("color.n.01"))

{"Davy's gray",
 "Davy's grey",
 'Indian red',
 'Paris green',
 'Prussian blue',
 'Turkey red',
 'Tyrian purple',
 'Vandyke brown',
 'Venetian red',
 'achromasia',
 'achromatic color',
 'achromatic colour',
 'alabaster',
 'alizarine red',
 'amber',
 'apatetic coloration',
 'aposematic coloration',
 'apricot',
 'aqua',
 'aquamarine',
 'ash gray',
 'ash grey',
 'azure',
 'beige',
 'black',
 'blackness',
 'bleach',
 'blond',
 'blonde',
 'blondness',
 'blue',
 'blue green',
 'blueness',
 'bluish green',
 'bone',
 'bottle green',
 'brick red',
 'brown',
 'brownish yellow',
 'brownness',
 'buff',
 'burgundy',
 'burnt sienna',
 'burnt umber',
 'canary',
 'canary yellow',
 'caramel',
 'caramel brown',
 'cardinal',
 'carmine',
 'carnation',
 'cerise',
 'cerulean',
 'chalk',
 'charcoal',
 'charcoal gray',
 'charcoal grey',
 'chartreuse',
 'cherry',
 'cherry red',
 'chestnut',
 'chocolate',
 'chromatic color',
 'chromatic colour',
 'chromatism',
 'chrome green',
 'chrome red',
 'claret',
 'coal b

In [10]:
def find_all_words_in_text(words, spacy_tokens):
    """ For a given set of words, find each instance among a list of tokens already
    processed by spacy.  Returns a list of token indexes that match.  (Note this only
    identifies single words, not multi-word phrases.)
    """
    all_matches=[]
    for idx, token in enumerate(spacy_tokens):
        if token.lemma_ in words:
            all_matches.append(idx)
    return all_matches

In [11]:
def print_concordance(matches, spacy_tokens, window=3):
    """ For a given set of token indexes, prints out a window of words around each match,
    in the style of a concordance.
    """
    
    RED="\x1b[31m"
    BLACK="\x1b[0m"
    
    spacing=window*10
    for match in matches:
        start=match-window
        end=match+window+1
        if start < 0:
            start=0
        if end > len(spacy_tokens):
            end=len(spacy_tokens)
        pre=' '.join([token.text for token in spacy_tokens[start:match]])
        post=' '.join([token.text for token in spacy_tokens[match+1:end]])
        print("%s %s%s%s %s" % (pre.rjust(spacing), RED, spacy_tokens[match].text, BLACK, post))

In [12]:
def read_text(filename):
    """ Read a text, replacing all whitespace sequences with a single space.
    """
    with open(filename, encoding="utf-8") as file:
        return re.sub("\s+", " ", file.read())

In [13]:
book=read_text("../data/pride_and_prejudice.txt")

In [14]:
spacy_tokens=nlp(book)

In [15]:
def wordnet_search(synset, spacy_tokens):
    """ This functions searchs through all of the tokens in the spacy_tokens argument to find
    any mention of words in the synset or any of its hyponyms.
    """
    targets=get_words_in_hypo(synset)
    matches=find_all_words_in_text(targets, spacy_tokens)
    print_concordance(matches, spacy_tokens)

Q1. Let's do a very coarse tagging of a document to find all of the mentions of a specific WordNet synset and all of its hyponyms. Using the functions above, find all of the color terms in *Pride and Prejudice*.

In [16]:
wordnet_search(wn.synset("color.n.01"), spacy_tokens)

                     he wore a blue coat , and
                    and rode a black horse . An
                   a bottle of wine a day .
                     I liked a red coat myself very
                  given to her complexion , and doubt
              till summoned to coffee . She was
                 walking , the tone of her voice
                   with a fine complexion and good -
                   , but their colour and shape ,
             Nicholls has made white soup enough ,
                        is _ a shade in a character
            reject the offered olive - branch .
                   idea of the olive - branch perhaps
                     come in a scarlet coat , and
                  in any other colour . As for
                 In a softened tone she declared herself
                . Both changed colour , one looked
                  , one looked white , the other
                   , the other red . Mr. Wickham
              , breathing port wine , who follo

Q2. Find all of the vehicles mentioned in *Pride and Prejudice*.

Q3. Find all of the verbs of speaking in *Pride and Prejudice*.

Q4. Find all of the people in *Pride and Prejudice*.

Q5. The methods above all identify *any* mentions of a WordNet synset in a text  -- e.g., every instance of *bank* would be identified as a hit for query bank.n.01 ("sloping land ..."), even if its specific word sense in context was the financial institution (or even a verb).  How might we improve on this method?